# Illinois Government Buildings Dashboard

Ben Ostermeier

## Import Libraries and Read Data

In [1]:
import pandas as pd
import bqplot
import numpy as np
import ipywidgets

On read, I set any incorrect zero values for square footage and years to not applicable

In [2]:
buildings = pd.read_csv("https://uiuc-ischool-dataviz.github.io/is445_spring2022/week03/data/building_inventory.csv", na_values={'Year Acquired':0, 'Year Constructed':0, 'Square Footage':0})

In [3]:
buildings.head()

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided


## Prepare Data

Next, I create a pivot table (buildings_pv) that is the mean of each combination of county and agency.

In [4]:
buildings_pv = pd.pivot_table(buildings, values='Square Footage', index=['Agency Name'],
                    columns=['County'], aggfunc=np.mean, fill_value=0)

In [5]:
buildings_pv.head()

County,Adams,Alexander,Bond,Boone,Brown,Bureau,Calhoun,Carroll,Cass,Champaign,...,Wabash,Warren,Washington,Wayne,White,Whiteside,Will,Williamson,Winnebago,Woodford
Agency Name,,,,,,,,,,,,,,,,,,,,,
Appellate Court / Fifth District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appellate Court / Fourth District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appellate Court / Second District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appellate Court / Third District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicago State University,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


I set any zeros to nan to allow for log10 and to make the colors more variable. When I didn't do this and included zero without log, it meant that most of the cells were at the "low" color, with only about two to three cells at very high.

In [6]:
# used https://stackoverflow.com/questions/49575897/cant-replace-0-to-nan-in-python-using-pandas to replace 0 with nan
buildings_pv.replace(0, np.nan, inplace=True)

In [7]:
buildings_pv.head(1)

County,Adams,Alexander,Bond,Boone,Brown,Bureau,Calhoun,Carroll,Cass,Champaign,...,Wabash,Warren,Washington,Wayne,White,Whiteside,Will,Williamson,Winnebago,Woodford
Agency Name,,,,,,,,,,,,,,,,,,,,,
Appellate Court / Fifth District,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Heatmap

In [8]:
# 1 set up label
myLabel = ipywidgets.Label()

#2 scales
col_sc = bqplot.ColorScale(scheme='YlGn')
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()

# used https://github.com/bqplot/bqplot/issues/1197 and
# https://bqplot.readthedocs.io/en/latest/_generate/bqplot.axes.Axis.html#bqplot.axes.Axis.tick_style
# to figure out setting the style of tick labels

# 3 axis
ax_col = bqplot.ColorAxis(scale=col_sc,
                        orientation='vertical',
                       side='right')
ax_x = bqplot.Axis(scale=x_sc, tick_rotate=90, 
                   tick_style={'font-size':'7px', 'text-anchor': 'start'}, offset={'scale':x_sc, 'value':'10px'})
ax_y = bqplot.Axis(scale=y_sc, orientation='vertical',tick_style={'font-size':'7px'})

# 4 Mark
heat_map = bqplot.GridHeatMap(color=np.log10(buildings_pv.values), 
                              row=buildings_pv.index,
                              column=buildings_pv.columns,
                              scales={'color':col_sc, 
                                                  'row':y_sc, 
                                                  'column':x_sc},
                             interactions={'click':'select'},
                             selected_style={'fill':'red'},
                             default_color='#060482')

# 5 interaction
def on_selected(change):
    if len(change['owner'].selected) == 1:
        i,j = change['owner'].selected[0]
        agency = buildings.loc[buildings['Agency Name'] == buildings_pv.index[i]]
        agency_county = agency.loc[agency['County'] == list(buildings_pv)[j]]
        data = agency_county.groupby('Year Acquired')["Square Footage"].sum()
        bar.x = data.index
        bar.y = data.values
        square_footage = buildings_pv.iloc[i].iloc[j]
        if (data.empty):
            myLabel.value = 'There are no ' + buildings_pv.index[i] + ' buildings in ' + list(buildings_pv)[j] + ' County'
        else:
            # used https://www.w3resource.com/python-exercises/string/python-data-type-string-exercise-32.php as a reminder on how to format floats
            myLabel.value = 'Mean Square Footage for ' + buildings_pv.index[i] + ' buildings in ' + list(buildings_pv)[j] + ' County: {:.0f}'.format(square_footage)
            

heat_map.observe(on_selected, 'selected')

# combine into a figure
# figured out fig margins using https://notebook.community/rmenegaux/bqplot/examples/Axis%20Properties
m_fig = dict(left=100, top=0, bottom=50, right=50)
fig = bqplot.Figure(marks=[heat_map], axes=[ax_col, ax_x, ax_y], fig_margin=m_fig)

## Histogram

In [9]:
i,j = 6,15 #hard code for now on cell with data

#2 scales
x_sch = bqplot.LinearScale()
y_sch = bqplot.LinearScale()

# 3 Axis
x_axh = bqplot.Axis(scale=x_sch, label='Year', tick_rotate=90, tick_style={'font-size':'9px', 'text-anchor': 'start'},
                    offset={'scale':x_sc, 'value':'10px'}, tick_format='d')
# figured out tick format uses python formating through https://bqplot.readthedocs.io/en/latest/_generate/bqplot.axes.Axis.html
# and reminded myself how to use it with https://www.w3schools.com/python/ref_string_format.asp. this avoids showing the square footage in
# scientific notation
y_axh = bqplot.Axis(scale=y_sch, label='Acquired Square Footage', orientation='vertical', tick_format='d')

agency = buildings.loc[buildings['Agency Name'] == buildings_pv.index[i]]
agency_county = agency.loc[agency['County'] == list(buildings_pv)[j]]
data = agency_county.groupby('Year Acquired')["Square Footage"].sum()

# 4 marks
# figured out color using documentation https://bqplot.readthedocs.io/en/latest/_generate/bqplot.marks.Bars.html
bar = bqplot.Bars(x=data.index, y=data.values, scales={"x": x_sch, "y": y_sch},  colors='#2d914c')
# 5 figure
# figured out label offset using https://notebook.community/rmenegaux/bqplot/examples/Axis%20Properties
m_figh = dict(left=75, top=0, bottom=50, right=0)
x_axh.label_offset = '3em'
y_axh.label_offset = '4em'
figh = bqplot.Figure(marks=[bar], axes=[x_axh,y_axh], fig_margin=m_figh)

## Combine

In [10]:
fig.layout.min_width='800px'
figh.layout.min_width='300px'
figures = ipywidgets.HBox([fig,figh])
myDashboard = ipywidgets.VBox([myLabel,figures])
myDashboard

Note a few design customizations that I implemented:

* Added margins to make sure the labels and charts fit properly
* Shrunk the text and rotated tick labels for both x-axies to fit the labels
* Decided to use green and yellow as the color scale. Originally I chose a color scale that did not involve white, since at the time I did not have log10 working and the non-data cells were white. I wanted to distinguish cells that had at least a little bit of data from those that had none.
* Added county and agency names to the top data label. This makes identifying the rows easier.
* Added feature to change the label if selecting a non-data cell, which says a more meaningful "no buildings" instead of nan.
* Ensured that data labels for the bar graph were in standard form instead of scientific notation. It doesn't make sense to use scientific notation for years, and the numbers for square feet aren't so large that scientific notation makes sense.

Here are a few features I would like to implement if I had more time:

* I spent a fair amount of time trying to change the 'no data' color to something less dramatic and imposing than black. Unfortunately I ran out of time. If I were to change the color, I would first try white to see if it was distinguishable enough from cells with small square footage. If not, I might try something completely different from the green-yellow color scheme to make it clearly different.
* I wasn't able to figure out how to space out the tick labels from the x-axis, since at this point it's too close in my view and off-center.
* I would also like to make the text of the label larger.
* If feasible, it would also be nice to have the same range of dates available on the x-axis of the bar graph, so that it would go from 1750s to 2019 for every cell. The downside though is that it migth make the data harder to read for most cells that are solely in the 20th and 21st centuries.